# Current code and solution

In [1]:
""" 
Optimization of the sky for OVRO scheduler

Walter Max-Moerbeck, Oct 30, 2023
Antonia Bravo Rojo, Dic 26, 2023
"""

#pip install import-ipynb
import ipynb.fs.full.sky_optimizer_with_GA as SO
#import ipynb.fs.full.sky_optimizer as SO

from datetime import date, timedelta
#from py40m.calendar import datetime_to_mjd
import pickle
import time

In [2]:
# observation parameters:
lst_start = 0.0
lst_int = 1.76
sun_jd = 60248.0 # Oct 30, 2023
date_tag = '30102023'

# Load the source and regions optimized in a previous stage
with open('region_optimizer_results.dat', 'rb') as file:
    sources = pickle.load(file)
    regions = pickle.load(file)
file.close()

N_iter = 0
total_minimum = 1.0e100
t0 = time.time()

initial_order, initial_total_time, initial_order_lst, lst_start = SO.order_regions_slew_time(regions, 
                                                                                             sources,
                                                                                             lst_start=lst_start,
                                                                                             lst_obs_win=5./60.,
                                                                                             za_t=0.0,
                                                                                             az_t=180.0,
                                                                                             sun_jd=sun_jd)


print ('initial best slew estimate with lst_start', lst_start, 'total_time', initial_total_time)
print ('with region order:', initial_order)

sun ra dec= 226.73344783529663 -18.081762056554513
region 0 sun_angle 123.83209778990762 34.3784 73.8256
region 1 sun_angle 112.3785012825411 111.549 79.1919
region 2 sun_angle 94.14420550695256 222.12 76.0197
region 3 sun_angle 104.5129271120882 301.379 77.8786
region 4 sun_angle 126.69864758755625 17.5536 68.0948
region 5 sun_angle 131.28849791986326 72.3471 63.5358
region 6 sun_angle 115.37960907919714 110.473 71.3433
region 7 sun_angle 101.20733062769396 149.697 65.565
region 8 sun_angle 81.90368428036096 219.191 63.6103
region 9 sun_angle 88.69122961656477 250.533 68.9442
region 10 sun_angle 100.66732805067508 301.87 66.1228
region 11 sun_angle 114.11535119514994 337.652 69.7744
region 12 sun_angle 140.9771585333635 18.3625 49.8067
region 13 sun_angle 135.44822347094774 41.2404 62.4685
region 14 sun_angle 133.91633138689156 85.3174 53.2067
region 15 sun_angle 118.58940699255784 113.469 50.3692
region 16 sun_angle 109.32029219899988 128.729 55.5728
region 17 sun_angle 90.8032946266

In [3]:
best_time = initial_total_time
best_regions = initial_order[:]

#puse un num random para que corriera
if best_time <= 1000:#length:
    N_iter = 0
else:
    N_iter = 0
    
for i in range(N_iter):

    updated_regions, updated_time, delta =\
                                        SO.modify_path_keep_cal(regions,
                                                       initial_order,
                                                       lst_start,
                                                       sources,
                                                       wait=True,
                                                       za_t=0.0,
                                                       az_t=180.0)

    delta = updated_time -initial_total_time
    if updated_time < initial_total_time:
        initial_order = updated_regions[:]
        initial_total_time = updated_time

    if initial_total_time < best_time:
        best_time = initial_total_time
        best_regions = updated_regions[:]#

    if i % 100 == 0:
        print ('for iter ', i, 'initial_time =', initial_total_time, 'updated_time =', updated_time, 'best_time=', best_time, 'delta=', delta)

t1 = time.time()
print ('Total time for solution  ', t1 - t0, ' sec')
print ('Total time for solution  ', (t1 - t0)/60., ' min')
print ('Total time for solution  ', (t1 - t0)/3600., ' hour')

Total time for solution   877.5067558288574  sec
Total time for solution   14.625112597147623  min
Total time for solution   0.24375187661912706  hour


In [4]:
# simulate the observation
report = SO.simulate_regions_final(regions,
                   best_regions,
                   lst_start,
                   sources,
                   wait=True)

za, az, t_obs, t_slew, t_wait, lst = SO.report_stats(report)

#take away the observing time from the last region which is just added to give the slew time and wait time
total = sum(t_obs) + sum(t_slew) + sum(t_wait)
print ('best obtained time =', total, 'with region order', best_regions)


--------------------------------------------------
Schedule summary
number lst za   az   t_obs  t_slew  t_wait
        h  deg  deg  h      h       h
53	0.11648	34.90	269.64	0.34009	0.11648	0.00000
54	0.79148	28.44	261.64	0.56339	0.02428	0.31063
39	1.69932	34.15	290.26	0.62817	0.01481	0.32963
142	2.64603	38.06	54.07	0.64981	0.31855	0.00000
15	3.32042	46.50	52.24	0.79681	0.02458	0.00000
29	4.13645	47.42	63.16	0.78166	0.01922	0.00000
61	4.93293	47.52	89.90	0.43898	0.01482	0.00000
76	5.38497	42.05	120.71	0.42950	0.01306	0.00000
92	5.83497	43.53	155.19	0.25296	0.02050	0.00000
131	6.11777	51.77	171.95	0.34454	0.02985	0.00000
116	6.47205	51.01	189.31	0.45954	0.00974	0.00000
75	6.95248	35.19	217.67	0.28546	0.02089	0.00000
91	7.27059	43.64	230.33	0.52338	0.03265	0.00000
141	7.83860	47.03	207.88	0.64981	0.04463	0.00000
115	8.51960	50.84	198.75	0.27692	0.03120	0.00000
130	8.81793	55.70	197.39	0.30118	0.02141	0.00000
114	9.13306	48.38	183.43	0.48901	0.01395	0.00000
129	9.63561	51.28	173.78	0.2901

In [5]:
#import ipynb.fs.full.sky_optimizer_with_GA as SO
order = ['26', '53', '38', '54', '27', '141', '91', '58', '116', '132', '59', '5', '130', '60', '76', '129', '115', '114', '128', '64', '112', '79', '127', '96', '80', '35', '37', '22', '108', '124', '83', '101', '69', '84', '123', '107', '122', '121', '52', '85', '106', '12', '0', '135', '119', '13', '41', '72', '74', '44', '75', '28', '7', '16', '14', '42', '6', '15', '1', '33', '66', '111', '126', '81', '82', '99', '109', '68', '137', '21', '49', '102', '86', '70', '55', '138', '24', '103', '87', '105', '139', '10', '142', '133', '89', '73', '90', '117', '29', '31', '30', '46', '95', '18', '94', '77', '45', '62', '63', '51', '47', '48', '19', '34', '50', '20', '39', '25', '3', '56', '104', '71', '88', '134', '118', '57', '40', '4', '61', '131', '92', '93', '17', '140', '113', '32', '136', '8', '65', '97', '98', '67', '36', '9', '2', '11']
order = SO.genetic_algorithm_sky(regions, sources, order, lst_start, 250, 0.8, 10)

optimum travel order =   ['26', '53', '38', '54', '27', '141', '91', '58', '116', '132', '59', '5', '130', '60', '76', '129', '115', '114', '128', '64', '112', '79', '127', '96', '80', '35', '37', '22', '108', '124', '83', '101', '69', '84', '123', '107', '122', '121', '52', '85', '106', '12', '0', '135', '119', '13', '41', '72', '74', '44', '75', '28', '7', '6', '14', '42', '16', '15', '1', '33', '66', '111', '126', '81', '82', '99', '109', '68', '137', '21', '49', '102', '86', '70', '55', '138', '24', '103', '87', '105', '139', '10', '142', '133', '89', '73', '90', '117', '29', '31', '30', '46', '95', '18', '94', '77', '45', '62', '63', '51', '47', '48', '19', '34', '50', '20', '39', '25', '3', '56', '104', '71', '88', '134', '118', '57', '40', '4', '61', '131', '92', '93', '17', '140', '113', '32', '136', '8', '65', '97', '98', '67', '36', '9', '2', '11']
estamos trabajando con n° de fuentes igual a  136
  

t_obs =  76.73017472270429
t_slew =  9.681653367589124
t_wait =  25.6493222

In [6]:
#t_obs =  76.2081301779091
#t_slew =  7.728744662986844
#t_wait =  4.7950508379228625

#t_obs =  75.76889887299201
#t_slew =  8.004048505114488
#t_wait =  4.958978380441745
#t_obs+t_slew+t_wait

In [7]:
order = SO.local_search_multi_objective_batch(regions, order, lst_start, sources, 250, 0.8, 10, 5)

optimum travel order =   ['26', '53', '38', '54', '27', '141', '91', '58', '116', '132', '59', '5', '130', '60', '76', '129', '115', '114', '128', '64', '112', '79', '127', '96', '80', '35', '37', '22', '108', '124', '83', '101', '69', '84', '123', '107', '122', '121', '52', '85', '106', '12', '0', '135', '119', '13', '41', '72', '74', '44', '75', '28', '7', '6', '14', '42', '16', '15', '1', '33', '66', '111', '126', '81', '82', '99', '109', '68', '137', '21', '49', '102', '86', '70', '55', '138', '24', '103', '87', '105', '139', '10', '142', '133', '89', '73', '90', '117', '29', '31', '30', '46', '95', '18', '94', '77', '45', '62', '63', '51', '47', '48', '19', '34', '50', '20', '39', '25', '3', '56', '104', '71', '88', '134', '118', '57', '40', '4', '61', '131', '92', '93', '17', '140', '113', '32', '136', '8', '65', '97', '98', '67', '36', '9', '2', '11']
estamos trabajando con n° de fuentes igual a  136


t_obs =  76.73017472270429
t_slew =  9.681653367589124
t_wait =  25.649322251

In [8]:
order = SO.multi_objective_local_search(regions, order, lst_start, sources, 250, 0.8, 10, 0.5)

optimum travel order =   ['13', '53', '38', '54', '4', '141', '57', '58', '116', '132', '59', '5', '130', '60', '76', '129', '115', '114', '128', '64', '112', '79', '96', '126', '80', '35', '37', '22', '124', '108', '83', '101', '69', '84', '123', '107', '122', '121', '52', '85', '106', '12', '0', '135', '119', '26', '27', '72', '74', '44', '75', '28', '7', '6', '41', '42', '14', '15', '1', '33', '66', '127', '111', '81', '82', '99', '109', '68', '137', '21', '49', '102', '86', '70', '55', '138', '24', '103', '87', '105', '139', '10', '142', '133', '89', '73', '91', '117', '29', '31', '30', '46', '95', '18', '94', '62', '77', '45', '63', '51', '47', '48', '19', '34', '50', '20', '39', '25', '3', '56', '104', '71', '88', '134', '118', '90', '40', '16', '61', '131', '92', '93', '17', '140', '113', '32', '136', '8', '65', '97', '98', '67', '36', '9', '2', '11']
estamos trabajando con n° de fuentes igual a  136


t_obs =  76.70228200253794
t_slew =  10.070964226760873
t_wait =  1.287439515

In [9]:
# simulate the observation
report = SO.simulate_regions_final(regions,
                   order,
                   lst_start,
                   sources,
                   wait=True)

za, az, t_obs, t_slew, t_wait, lst = SO.report_stats(report)

#take away the observing time from the last region which is just added to give the slew time and wait time
total = sum(t_obs) + sum(t_slew) + sum(t_wait)
print ('best obtained time =', total)


--------------------------------------------------
Schedule summary
number lst za   az   t_obs  t_slew  t_wait
        h  deg  deg  h      h       h
13	0.19457	35.58	31.58	0.30786	0.19457	0.00000
53	0.69140	40.89	-86.04	0.77251	0.18896	0.00000
38	1.49259	39.15	-66.12	0.31455	0.02868	0.00000
54	1.85658	40.83	-87.90	0.98589	0.04944	0.00000
4	2.91321	33.86	-16.49	0.35755	0.07074	0.00000
141	3.70388	59.49	126.67	0.64981	0.18477	0.24835
57	4.48857	37.33	243.12	0.60495	0.13488	0.00000
58	5.12543	26.25	242.68	0.36832	0.02189	0.01003
116	5.58795	50.97	171.09	0.45674	0.09420	0.00000
132	6.06966	59.51	203.66	0.31963	0.02498	0.00000
59	6.43085	24.74	217.85	0.42201	0.04156	0.00000
5	6.99248	32.06	334.82	1.02166	0.13962	0.00000
130	8.20667	53.98	183.79	0.30101	0.19252	0.00000
60	8.58507	28.26	230.87	0.30154	0.07739	0.00000
76	8.94178	31.05	213.72	0.46565	0.05517	0.00000
129	9.48582	51.65	169.81	0.28991	0.07839	0.00000
115	9.82581	58.44	220.06	0.27510	0.05008	0.00000
114	10.14503	50.99	202.11	0.487

In [10]:
# if this solution is best save with starndard name
#if total < total_minimum:
#    SO.save_results(best_regions, lst, sky_optimization_results_file)


# Save the results of sky optimization. 
#SO.save_results(sorted_regions, sorted_regions_lst, sky_optimization_results_file)

# simulate the observation
#report = SO.simulate_regions_observation(regions,
#                                         sorted_regions,
#                                         lst_start,
#                                         sources,
#                                         wait=True)

#za, az, t_obs, t_slew, t_wait = SO.report_stats(report)


#file = open(sky_optimization_results_file, 'w')
#pickle.dump(best_regions, file)
#pickle.dump(lst, file)
#file.close()